# Installatation

In [ ]:
!pip install langchain-community pypdf

In [1]:
!pip install python-dotenv


In [12]:
!pip install huggingface

In [19]:
!pip install -qU langchain-huggingface

In [17]:
!pip install sentence-transformers

  Using cached scipy-1.16.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached markupsafe-3.0.3-cp311-cp311-win_amd64.whl.metadata (2.8 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.0 MB 4.2 MB/s eta 0:00:03
   ----- ---------------------------------- 1.6/12.0 MB 4.2 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/12.0 MB 4.2 MB/s eta 0:00:03
   --------- ------------------------------ 2.9/12.0 MB 3.8 MB/s eta 0:00:03
   ------------- -------------------------- 3.9/12.0 MB 4.0 MB/s eta 0:00:03
   ---------------- ----------------------- 5.0/12.0 MB 4.1 MB/s eta 0:00:02
   ------------------- -------------------- 5.8/12.0 MB 4.1 MB/s

# Connect to langsmith

load .env file

In [1]:
from dotenv import load_dotenv
load_dotenv()


True

double check whether loading succeed or not

In [2]:
import os
print(os.environ.get("LANGSMITH_TRACING"))
print(os.environ.get("LANGSMITH_API_KEY")[:8])


true
lsv2_pt_


In [3]:
import os
# 选择终端为EU region
os.environ["LANGSMITH_ENDPOINT"] = "https://eu.api.smith.langchain.com"
# 可选：给项目起名，方便在 UI 里找
os.environ["LANGSMITH_PROJECT"] = "test-tracing"

In [4]:
import os
print("TRACING =", os.environ.get("LANGSMITH_TRACING"))
print("KEY prefix =", (os.environ.get("LANGSMITH_API_KEY") or "")[:8])
print("ENDPOINT =", os.environ.get("LANGSMITH_ENDPOINT"))
print("PROJECT =", os.environ.get("LANGSMITH_PROJECT"))


TRACING = true
KEY prefix = lsv2_pt_
ENDPOINT = https://eu.api.smith.langchain.com
PROJECT = test-tracing


check whether succeed

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

response = llm.invoke("用一句话解释什么是 dataset redundancy")
print(response.content)


数据集冗余是指在数据集中存在重复或多余的信息，导致数据存储和处理变得低效。


# 1. Documents and Document Loaders

LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has three attributes:<br>

- page_content: a string representing the content;<br>

- metadata: a dict containing arbitrary metadata;<br>

- id: (optional) a string identifier for the document.<br>

The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.<br>
We can generate sample documents when desired:<br>

In [4]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]


## Loading documents

In [5]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../example_data/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


PyPDFLoader loads one Document object per PDF page. For each, we can easily access:<br>
1. The string content of the page;
2. Metadata containing the file name and page number.

In [8]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '../example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}



## Splitting

We can use text splitters for this purpose. Here we will use a simple text splitter that partitions based on characters. We will split our documents into chunks of 1000 characters with 200 characters of overlap between chunks. The overlap helps mitigate the possibility of separating a statement from important context related to it.

 We use the **RecursiveCharacterTextSplitter**, which will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases.


We set **add_start_index=True** so that the character index where each split Document starts within the initial Document is preserved as metadata attribute “start_index”.

在 LangChain 里，split_documents 返回的是 Document 对象列表，每个元素都有两个核心属性：

- page_content

- metadata

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

516


In [7]:
for i, doc in enumerate(all_splits[:5]):  # 只看前5个
    print(f"Chunk {i}")
    print("Metadata:", doc.metadata)
    print("Content preview:", doc.page_content[:200])
    print("=" * 60)


Chunk 0
Metadata: {'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '../example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1', 'start_index': 0}
Content preview: Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F
Chunk 1
Metadata: {'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financi

# 2. Embeddings

Vector search is a common way to store and search over unstructured data (such as unstructured text). The idea is to store numeric vectors that are associated with the text. Given a query, we can **embed** it as a vector of the same dimension and use vector similarity metrics (such as cosine similarity) to identify related text.


LangChain supports embeddings from dozens of providers. These models specify how text should be converted into a numeric vector. Let’s select a model:

In [ ]:
# I choose to use huggingface here

# check existing models
from huggingface_hub import scan_cache_dir

cache_info = scan_cache_dir()
for repo in cache_info.repos:
    print(repo.repo_id)


c:\Users\15509\anaconda3\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


albert/albert-base-v2
sasha/regardv3
xlm-roberta-base
sentence-transformers/all-mpnet-base-v2
cardiffnlp/twitter-roberta-base-sentiment-latest


In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [8]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

# assert表示：我“确信” vector_1 和 vector_2 的长度必须相等
# 如果不相等，程序立刻报错并停止
assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[0.04747222736477852, 0.02167578600347042, -0.00901806727051735, 0.005356725305318832, 0.02555767446756363, -0.01023025717586279, -0.008414025418460369, 0.039303913712501526, 0.021570533514022827, -0.0240954477339983]


# 3. Vector stores

LangChain **VectorStore** objects contain methods for adding text and Document objects to the store, and querying them using various similarity metrics. They are often initialized with **embedding models**, which determine how text data is translated to numeric vectors.

In [ ]:
!pip install -U "langchain-core"

In [10]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

Having instantiated our vector store, we can now index the documents.

In [11]:
ids = vector_store.add_documents(documents=all_splits)

ids 的结构是：
```python
List[str]

```
也就是：
```python
['uuid-1', 'uuid-2', 'uuid-3', ...]
```

每一个 id 对应一个 Document / 一个 embedding 向量

在 InMemoryVectorStore 里：

如果你 没有显式提供 ids

LangChain 会自动：

```text
为每个 Document 生成一个唯一字符串 ID
```


In [14]:
print(ids[:5])

['0c56ddd8-c40a-47bd-bb0b-744dc80fbbb4', '02e8b5c9-58a8-4e8f-888e-1240ee9d1660', '17e751cc-b34d-4082-935b-dca9b151d2a8', 'a31091ec-1847-4c43-a18c-181481808491', '174b4095-aef4-4871-941c-3e86e25e19f3']


In [16]:
# 自己控制id
custom_ids = [
    f"{doc.metadata.get('source', 'doc')}_{doc.metadata.get('start_index')}"
    for doc in all_splits
]

ids = vector_store.add_documents(
    documents=all_splits,
    ids=custom_ids
)


In [17]:
print(ids[:5])

['../example_data/nke-10k-2023.pdf_0', '../example_data/nke-10k-2023.pdf_781', '../example_data/nke-10k-2023.pdf_1592', '../example_data/nke-10k-2023.pdf_2508', '../example_data/nke-10k-2023.pdf_3276']


## Usage

Embeddings typically represent text as a “dense” vector such that texts with similar meanings are geometrically close.

This lets us retrieve relevant information just by passing in a question, without knowledge of any specific key-terms used in the document.

In [ ]:
# Return documents based on similarity to a string query:

# 检索可能包含答案的文本片段
# results = List[Document]
# choose the top k most similar chunk (default k is 4)
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 

In [21]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"
"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, equipment, accessories and services. NIKE is
the largest seller of athletic footwear and apparel in the world. We sell our products through NIKE Direct operations, which are comprised of both NIKE-owned retail stores
and sales through our digital platforms (also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'cr

Return scores:

In [ ]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")

# results 不是 Document 列表
# 而是一个 二元组列表
# results = [
#     (Document(...), score_0),
#     (Document(...), score_1),
#     ...
# ]
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.8137388177528481

page_content='Table of Contents
YEAR ENDED MAY 31,
(Dollars in millions) 2023 2022 2021
REVENUES
North America $ 21,608 $ 18,353 $ 17,179 
Europe, Middle East & Africa 13,418 12,479 11,456 
Greater China 7,248 7,547 8,290 
Asia Pacific & Latin America 6,431 5,955 5,343 
Global Brand Divisions 58 102 25 
Total NIKE Brand 48,763 44,436 42,293 
Converse 2,427 2,346 2,205 
Corporate 27 (72) 40 
TOTAL NIKE, INC. REVENUES $ 51,217 $ 46,710 $ 44,538 
EARNINGS BEFORE INTEREST AND TAXES
North America $ 5,454 $ 5,114 $ 5,089 
Europe, Middle East & Africa 3,531 3,293 2,435 
Greater China 2,283 2,365 3,243 
Asia Pacific & Latin America 1,932 1,896 1,530 
Global Brand Divisions (4,841) (4,262) (3,656)
Converse 676 669 543 
Corporate (2,840) (2,219) (2,261)
Interest expense (income), net (6) 205 262 
TOTAL NIKE, INC. INCOME BEFORE INCOME TAXES $ 6,201 $ 6,651 $ 6,661 
ADDITIONS TO PROPERTY, PLANT AND EQUIPMENT
North America $ 283 $ 146 $ 98 
Europe, Middle East & Africa 21

In [23]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Table of Contents
GROSS MARGIN
FISCAL 2023 COMPARED TO FISCAL 2022
For fiscal 2023, our consolidated gross profit increased 4% to $22,292 million compared to $21,479 million for fiscal 2022. Gross margin decreased 250 basis points to
43.5% for fiscal 2023 compared to 46.0% for fiscal 2022 due to the following:
*Wholesale equivalent
The decrease in gross margin for fiscal 2023 was primarily due to:
• Higher NIKE Brand product costs, on a wholesale equivalent basis, primarily due to higher input costs and elevated inbound freight and logistics costs as well as
product mix;
• Lower margin in our NIKE Direct business, driven by higher promotional activity to liquidate inventory in the current period compared to lower promotional activity in
the prior period resulting from lower available inventory supply;
• Unfavorable changes in net foreign currency exchange rates, including hedges; and
• Lower off-price margin, on a wholesale equivalent basis.
This was partially offset by:'

# 4. Retrievers

LangChain VectorStore objects do not subclass **Runnable**. 

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations). 

Although we can construct retrievers from vector stores, retrievers can interface with non-vector store sources of data, as well (such as external APIs).

### 一句话直觉版（先给结论）

**Runnable = “可以被统一调用、组合、并行、异步执行的 LangChain 组件”**
是一个有input,有output的函数！

换句话说：

**Runnable 是 LangChain 世界里的“标准算子接口”**

### 一、Runnable 在 LangChain 中是什么意思？

在 LangChain 里，Runnable 是一个 抽象基类 / 协议，表示一个对象：

- 可以被 调用（invoke）

- 可以 异步调用（ainvoke）

- 可以 批量调用（batch / abatch）

- 可以被 串联、并行、组合成 pipeline

也就是说，只要是 Runnable，LangChain 就知道：

```text
我该怎么用统一方式调用你
```
### 二、Runnable 具体提供了什么能力？

一个 Runnable 至少支持这些标准方法（概念上）：

- invoke(input)        # 同步单输入
- ainvoke(input)       # 异步单输入
- batch(inputs)        # 同步批量
- abatch(inputs)       # 异步批量


这就是你看到那句话里说的：

“they implement a standard set of methods”

### 三、为什么 VectorStore 不是 Runnable？

VectorStore 的本质是：

**数据结构 / 存储层**

它负责的是：

- 存 embedding

- 做 similarity search

- 管理向量索引

但它 不是一个“从 input → output 的标准算子”

所以你调用它时是：
```python
vector_store.similarity_search(query)
```

而不是：
```python
vector_store.invoke(query)  # ❌ 不存在
```

### 四、那为什么 Retriever 是 Runnable？
Retriever 的本质是：

**Query → Documents 的“计算过程”**

也就是说：

**输入**：query（字符串）
**输出**：documents（List[Document]）


这是一个标准函数形态，非常适合做成 Runnable。

所以：
```python
retriever = vector_store.as_retriever()
```

你得到的是一个：
```python
Runnable[str → List[Document]]
```

于是你可以：
```python
docs = retriever.invoke("When was Nike incorporated?")
```

也可以：
```python
docs = await retriever.ainvoke("...")
```
### 五、Runnable 的真正价值：可组合性

这是最重要的一点。

只要是 Runnable，就可以：

1️⃣ 串联（pipe）
```python
retriever | llm
```
2️⃣ 组成完整 RAG 链
```python
retriever | prompt | llm
```
3️⃣ 并行 / batch
```python
retriever.batch(queries)
```

👉 VectorStore 做不到这些

### 六、用一句类比（非常重要）

你可以这样理解：

| 概念          | 类比          |
| ----------- | ----------- |
| VectorStore | 数据库         |
| Retriever   | 查询函数        |
| Runnable    | 统一的“函数接口标准” |
| Chain       | 函数复合        |

所以那句话本质是在说：

**VectorStore 是“库”，Retriever 是“函数”**

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


# Decoration @chain 把一个 普通 Python 函数
# 转换成一个 LangChain Runnable（Chain）
# 换句话说：
# 普通函数  →  Runnable[str → List[Document]]

# 它不仅能被调用，还能：
# .invoke()
# .ainvoke()
# .batch()
# .abatch()
# | 管道组合
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)

# Return List[List[Document]]
retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='6f48bce7-4fae-4cdc-8d9c-5c4ba9773c5b', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '../example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 26, 'page_label': '27', 'start_index': 804}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned an

### vector_store.as_retriever(...)

这一步在做什么？

- 不复制数据

- 不重新建索引

- 只是创建一个“包装器”

可以理解为：

VectorStore (数据 + 搜索能力)
        ↓ 包装
Retriever (Runnable: query → documents)

### search_type="similarity"

指定 检索策略

最常见的几种是：

| search_type                    | 含义            |
| ------------------------------ | ------------- |
| `"similarity"`                 | 纯相似度搜索（默认）    |
| `"mmr"`                        | 最大边际相关性（减少冗余） |
| `"similarity_score_threshold"` | 相似度 + 阈值过滤    |


### search_kwargs={"k": 1}

传给底层 similarity_search 的参数。

等价于：
```python
vector_store.similarity_search(query, k=1)
```

也就是说：

每次 query

只返回 1 个 Document

In [27]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='6f48bce7-4fae-4cdc-8d9c-5c4ba9773c5b', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '../example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 26, 'page_label': '27', 'start_index': 804}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned an